In [1]:
!pip install MetaTrader5
!pip install yfinance --upgrade --no-cache-dir
!pip install get-all-tickers #https://github.com/shilewenuw/get_all_tickers/tree/master/get_all_tickers
!pip install html5lib
!pip install binance

ERROR: Invalid requirement: '#https://github.com/shilewenuw/get_all_tickers/tree/master/get_all_tickers'


# Desafio Final - I2A2 Financial Market
##### Gabriel Pehls, 12/01/2022

- Devemos montar uma análise do momentum do mercado;

## Requisitos:

- Análise das classificações do ativo, como:
    * Tendência do ativo (LTA/LTB/Lateralizando)
    * Suporte/Resistência
    * Formação de Candle (escolher algumas)
    * Médias móveis (9, 21)
    * Willians %
    * RSI
    * MACD
    * Bollinger Bands
- Utilizar o último mês de dados do ativo

In [2]:
from src.exchanges.binance import Binance
from src.exchanges.marketdata import marketdata

from src.trader import Trader

from utils.utils import get_secrets, get_settings
from src.directory_utilities import get_json_from_file

from src.indicators import Indicators
from datetime import datetime, timedelta
from binance.client import Client
import logging
import pandas as pd
import numpy as np

from src.graphical_analysis import analyzer, generate_graph

In [3]:
secrets = get_secrets()
settings = get_settings()

_trader = Trader(secrets, settings, operator = marketdata)

#_coin_pair = settings.get('coin_pair')
_coin_pair = "AAPL"

* O código foi montado com todos os indicadores concentrados no arquivo "indicadores.py", podendo ser modificado conforme a necessidade;
* A classe marketdata contém o fluxo de download dos dados, usando o yfinance, e uma modificação para encaixar em um formato comum com outras classes de recuperação de dados (como da binance, de criptoativos)
* Ao passarmos o mesmo como argumento do Trader (que é a nossa "classe-mestre"), o mesmo consultará os dados usando a interface de exchanges em comum entre todas (bittrex, binance e marketdata);
* O Trader recupera todos os indicadores na implementação da get_historical_prices, retornando os DFs que utilizamos para interpretação no analyzer;
* O Analyzer, por si só, "monta" as mensagens que precisamos, traduzindo o gráfico em palavras mais simples, recuperando o snapshot que é o objetivo do trabalho.

Acompanhemos:

In [4]:
%%time
df, crossovers, ma_crossovers, bb_crossovers, hammers, suportes, resistencias, high_trend, low_trend, close_trend = _trader.get_historical_prices(_coin_pair)

[*********************100%***********************]  1 of 1 completed
Wall time: 4.95 s


In [5]:
df2 = analyzer(_coin_pair).translate_graph_fromdf(df, crossovers, ma_crossovers, bb_crossovers, hammers, suportes, resistencias, high_trend, low_trend, close_trend)

In [6]:
df2[['Datetime','translated_message']]

,Datetime,translated_message
0,2021-01-11 21:00:00,"O ativo AAPL, na data de 11/01/2021, ás 21:00:..."
1,2021-01-12 21:00:00,"O ativo AAPL, na data de 12/01/2021, ás 21:00:..."
2,2021-01-13 21:00:00,"O ativo AAPL, na data de 13/01/2021, ás 21:00:..."
3,2021-01-14 21:00:00,"O ativo AAPL, na data de 14/01/2021, ás 21:00:..."
4,2021-01-18 21:00:00,"O ativo AAPL, na data de 18/01/2021, ás 21:00:..."
...,...,...
248,2022-01-04 21:00:00,"O ativo AAPL, na data de 04/01/2022, ás 21:00:..."
249,2022-01-05 21:00:00,"O ativo AAPL, na data de 05/01/2022, ás 21:00:..."
250,2022-01-06 21:00:00,"O ativo AAPL, na data de 06/01/2022, ás 21:00:..."
251,2022-01-09 21:00:00,"O ativo AAPL, na data de 09/01/2022, ás 21:00:..."


In [8]:
print(df2.iloc[21].translated_message)

O ativo AAPL, na data de 10/02/2021, ás 21:00:00: 
- Possui média móvel exponencial de 9 períodos ($135.1) acima da média simples de 21 períodos ($134.49),  O último cruzamento aconteceu 1 dias atrás, representando um Sinal de Compra (Long).
- Possui o oscilador RSI em uma área neutra (52.78)
- O último cruzamento do MACD foi em 01/02/2021, ás 21:00:00, 9 dias atrás, e foi um Sinal de Venda (Short)




In [9]:
generate_graph(df, crossovers)

---
#### Para múltiplos ativos:

In [15]:
for _coin_pair in ['AAPL','BABA']:
    df, crossovers, ma_crossovers, bb_crossovers, hammers, suportes, resistencias, high_trend, low_trend, close_trend = _trader.get_historical_prices(_coin_pair)
    df = analyzer(_coin_pair).translate_graph_fromdf(df, crossovers, ma_crossovers, bb_crossovers, hammers, suportes, resistencias, high_trend, low_trend, close_trend)
    print(df.tail(1).reset_index().iloc[0].translated_message)

[*********************100%***********************]  1 of 1 completed
O ativo AAPL, na data de 10/01/2022, ás 21:00:00: 
- Possui média móvel exponencial de 9 períodos ($174.99) abaixo da média simples de 21 períodos ($175.73),  O último cruzamento aconteceu 4 dias atrás, representando um Sinal de Venda (Short).
- Possui o oscilador RSI em uma área neutra (54.04)
- O último suporte foi em 03/10/2021, ás 21:00:00 e o fechamento atual ($175.08) é 26.62% maior que o último suporte ($138.27)
- A última resistência foi em 03/01/2022, ás 21:00:00 e o fechamento atual ($175.08) é 4.3% menor que a última resistência ($182.94)
- O último cruzamento do MACD foi em 16/12/2021, ás 21:00:00, 25 dias atrás, e foi um Sinal de Venda (Short)
- O preço de fechamento está em ($175.08) e a média móvel simples (de 20 períodos, ou banda do meio das Bandas de Bollinger) em ($175.73)
- O último toque nas Bandas de Bollinger aconteceu 32 dias atrás, onde o preço de fechamento era ($179.45), com Sinal de Venda (